In [1]:
import pandas as pd
import numpy as np
import os
from utils import *

In [2]:
DIR = r"C:\Users\kabir\Documents\R Projects\data"
assert os.path.exists(DIR)

2016 local data had a variety of variables not included in previous years' csvs. As such, these columns were removed. However, these variables are noted here for future reference:
COD_TIPO_ELEICAO, 
NOME_TIPO_ELEICAO, 
COD_ELEICAO, 
DATA_ELEICAO, 
ABRANGENCIA, 
NOME_UE, 
NOME_SOCIAL_CANDIDATO, 
TIPO_AGREMIACAO, 
TRANSITO. Such column names were not noted in the documentation.

In [3]:
folder_paths = os.listdir(DIR)
local_folder_paths = [f for f in folder_paths if "local" in f]
federal_folder_paths = [f for f in folder_paths if "fed" in f]
print(local_folder_paths)
print(federal_folder_paths)

['2004_local', '2008_local', '2012_local', '2016_local']
['2010_fed', '2014_fed']


In [4]:
def create_df(folder_path):
    path = os.path.join(DIR, folder_path)
    data_path = os.path.join(path, "electoral_data.csv")
    df = pd.read_csv(data_path)
    
    # essentially re-index by municipality-name combination
    local_ordered_cols = get_ordered_cols("local")
    df = df.reindex(columns = local_ordered_cols)
    
    # handle capitalization discrepancies so that all descriptions are treated equally
    df["DESCRICAO_CARGO"] = df["DESCRICAO_CARGO"].str.upper()
    
    # re-define df such that it only contains positions we are interested in analyzing further
    # also got rid of some elections that took place in local non-election years i.e. 2009 and 2011
    local_positions = get_positions("local")
    df = df[df.DESCRICAO_CARGO.isin(local_positions)]

    return df

In [5]:
local_df = pd.concat(map(create_df, local_folder_paths))

C:\Users\kabir\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\kabir\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [6]:
local_df["ANO_ELEICAO"].unique()

array([2004, 2008, 2012, 2016], dtype=int64)

In [7]:
# shape checked with manually summing row counts in each file, checks out
local_df.shape

(3132164, 26)

In [8]:
local_df["ANO_ELEICAO"].unique()

array([2004, 2008, 2012, 2016], dtype=int64)

In [9]:
pd.set_option("display.max_columns", None)

In [10]:
local_df.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
0,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,13,6.0,JOÃO DE ALMEIDA,JANGO,PREFEITO,-1,#NE#,2,DEFERIDO,4,NÃO ELEITO,13,PT,PARTIDO DOS TRABALHADORES,5.0,FRENTE POPULAR DE ACRELÂNDIA,#NE#,1090
1,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,14,1.0,PAULO CÉSAR FERREIRA DE ARAUJO,PAULINHO,PREFEITO,-1,#NE#,2,DEFERIDO,4,NÃO ELEITO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,1.0,#NULO#,#NE#,2060
2,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,45,27.0,SEBASTIÃO BOCALOM RODRIGUES,TIÃO BOCALOM,PREFEITO,-1,#NE#,2,DEFERIDO,1,ELEITO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,12.0,CONTINUANDO O PROGRESSO,#NE#,2173
3,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,11111,38.0,PAUMERIM LUIZ BERTO,PAUMERIM,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,11,PP,PARTIDO PROGRESSISTA,18.0,CONTINUANDO O PROGRESSO II,#NE#,178
4,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,12300,30.0,FRANCISCO ANTONIO DA SILVA DIAS,PROFESSOR FRANCISCO,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,16.0,CONTINUANDO O PROGRESSO I,#NE#,20


In [11]:
local_df["NOME_MUNICIPIO"].value_counts()

RIO DE JANEIRO       419207
SÃO PAULO            191947
BELO HORIZONTE        77980
SALVADOR              73856
MANAUS                45918
                      ...  
BETANIA DO PIAUI         14
JOAO DIAS                13
NOVA MARILANDIA          13
ENEAS MARQUES            12
ALEGRETE DO PIAUI        10
Name: NOME_MUNICIPIO, Length: 7681, dtype: int64

In [12]:
local_df["DESCRICAO_CARGO"].value_counts()
# city councilor
# mayor

VEREADOR    3047909
PREFEITO      84255
Name: DESCRICAO_CARGO, dtype: int64

In [13]:
# split into two dataframes, one for city councilors and one for mayors
local_df_city_councilors = local_df[local_df["DESCRICAO_CARGO"]=="VEREADOR"]
local_df_mayors = local_df[local_df["DESCRICAO_CARGO"]=="PREFEITO"]

In [14]:
print(local_df_city_councilors["DESCRICAO_CARGO"].value_counts())
print(local_df_mayors["DESCRICAO_CARGO"].value_counts())

VEREADOR    3047909
Name: DESCRICAO_CARGO, dtype: int64
PREFEITO    84255
Name: DESCRICAO_CARGO, dtype: int64


In [15]:
local_df_city_councilors.to_csv("city_councilors.csv", encoding='utf-8-sig', index=False)
local_df_mayors.to_csv("mayors.csv", encoding='utf-8-sig', index=False)

In [16]:
salvador_df = local_df[local_df["NOME_MUNICIPIO"] == "SALVADOR"]
cleaned_df = salvador_df[salvador_df["ANO_ELEICAO"] == 2008]
cleaned_df

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
44908,SALVADOR,2008,1,ELEIÇÕES 2008,BA,38490,38490,11,25,2357.0,ANTONIO CARLOS PEIXOTO DE MAGALHÃES NETO,ACM NETO,PREFEITO,12,APTO,2,DEFERIDO,4,NÃO ELEITO,25,DEM,DEMOCRATAS,202.0,A VOZ DO POVO,#NE#,17598
44909,SALVADOR,2008,1,ELEIÇÕES 2008,BA,38490,38490,11,50,7204.0,HILTON BARROS COELHO,HILTON COELHO,PREFEITO,12,APTO,2,DEFERIDO,4,NÃO ELEITO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,644.0,FRENTE DE ESQUERDA SOCIALISTA,#NE#,1628
44910,SALVADOR,2008,1,ELEIÇÕES 2008,BA,38490,38490,11,45,8082.0,ANTONIO JOSÉ IMBASSAHY DA SILVA,IMBASSAHY,PREFEITO,12,APTO,2,DEFERIDO,4,NÃO ELEITO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,789.0,PARA MELHORAR SALVADOR,#NE#,6159
44911,SALVADOR,2008,1,ELEIÇÕES 2008,BA,38490,38490,11,15,33306.0,JOÃO HENRIQUE DE BARRADAS CARNEIRO,JOÃO,PREFEITO,12,APTO,2,DEFERIDO,6,2º TURNO,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,3153.0,A FORÇA DO BRASIL EM SALVADOR,#NE#,13589
44912,SALVADOR,2008,1,ELEIÇÕES 2008,BA,38490,38490,11,13,1998.0,WALTER DE FREITAS PINHEIRO,PINHEIRO,PREFEITO,12,APTO,2,DEFERIDO,6,2º TURNO,13,PT,PARTIDO DOS TRABALHADORES,169.0,"SALVADOR, BAHIA, BRASIL",#NE#,13194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66006,SALVADOR,2008,2,ELEIÇÕES 2008,BA,38490,38490,11,13,1998.0,WALTER DE FREITAS PINHEIRO,PINHEIRO,PREFEITO,12,APTO,2,DEFERIDO,4,NÃO ELEITO,13,PT,PARTIDO DOS TRABALHADORES,169.0,#NULO#,#NE#,26910
66007,SALVADOR,2008,2,ELEIÇÕES 2008,BA,38490,38490,11,15,33306.0,JOÃO HENRIQUE DE BARRADAS CARNEIRO,JOÃO,PREFEITO,12,APTO,2,DEFERIDO,1,ELEITO,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,3153.0,#NULO#,#NE#,45877
66008,SALVADOR,2008,2,ELEIÇÕES 2008,BA,38490,38490,11,13,1998.0,WALTER DE FREITAS PINHEIRO,PINHEIRO,PREFEITO,12,APTO,2,DEFERIDO,4,NÃO ELEITO,13,PT,PARTIDO DOS TRABALHADORES,169.0,#NULO#,#NE#,36590
66009,SALVADOR,2008,2,ELEIÇÕES 2008,BA,38490,38490,11,15,33306.0,JOÃO HENRIQUE DE BARRADAS CARNEIRO,JOÃO,PREFEITO,12,APTO,2,DEFERIDO,1,ELEITO,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,3153.0,#NULO#,#NE#,34905


In [17]:
cleaned_df["NOME_CANDIDATO"].value_counts()

WALTER DE FREITAS PINHEIRO            40
ANTONIO CARLOS SILVA SANTOS           40
JOÃO HENRIQUE DE BARRADAS CARNEIRO    40
JOÃO ALFREDO REIS SILVA               20
JORGE BARBOSA SOUZA                   20
                                      ..
PAULO JOSE OLIVEIRA DOS SANTOS         8
GERALDO DE ALMEIDA BORGES              8
VALDIRA REIS DOS REIS                  4
RICARDO ANACLETO SANTOS                4
MARIVALDO DA SILVA CERQUEIRA           1
Name: NOME_CANDIDATO, Length: 822, dtype: int64

In [18]:
cleaned_df["TOTAL_VOTOS"].value_counts()

2        805
1        802
3        792
4        697
5        616
        ... 
1915       1
24440      1
1899       1
22265      1
1243       1
Name: TOTAL_VOTOS, Length: 911, dtype: int64

In [19]:
local_df["ANO_ELEICAO"].iloc[0]==2008

False

In [20]:
local_df[local_df["NOME_CANDIDATO"].str.contains("DA SILVA")]

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
4,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,12300,3.000000e+01,FRANCISCO ANTONIO DA SILVA DIAS,PROFESSOR FRANCISCO,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,1.600000e+01,CONTINUANDO O PROGRESSO I,#NE#,20
5,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,12333,3.100000e+01,MARIA ALVES DA SILVA QUEIROZ,NENE DO POSTO,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,1.600000e+01,CONTINUANDO O PROGRESSO I,#NE#,38
7,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,13000,2.600000e+01,JOSÉ IDNILTON SANTOS DA SILVA,PINGO,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,13,PT,PARTIDO DOS TRABALHADORES,1.000000e+01,#NULO#,#NE#,99
14,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,13613,1.800000e+01,APARECIDO SABINO DA SILVA,APARECIDO,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,13,PT,PARTIDO DOS TRABALHADORES,1.000000e+01,#NULO#,#NE#,115
24,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,13,23456,4.800000e+01,IVAN DA SILVA FERREIRA,IVAN,VEREADOR,-1,#NE#,2,DEFERIDO,2,SUPLENTE,23,PPS,PARTIDO POPULAR SOCIALISTA,1.800000e+01,CONTINUANDO O PROGRESSO II,#NE#,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983161,RECURSOLÂNDIA,2016,1,ELEIÇÕES MUNICIPAIS 2016,TO,73229,73229,13,43255,2.700000e+11,MARIA EUNICE CARNEIRO DA SILVA RIBEIRO,MARIA EUNICE,VEREADOR,2,DEFERIDO,12,APTO,5,SUPLENTE,43,PV,Partido Verde,2.700000e+11,NOVA RECURSOLANDIA,PT / PTB / PMDB / PTN / DEM / PV / PSD / SD,6
983166,CARMOLÂNDIA,2016,1,ELEIÇÕES MUNICIPAIS 2016,TO,73326,73326,13,25111,2.700000e+11,MARIA GERCIANE FRANCISCA DA SILVA,GERCIANE,VEREADOR,2,DEFERIDO,12,APTO,5,SUPLENTE,25,DEM,Democratas,2.700000e+11,CONTINUAR E PRECISO I,PSD / DEM / PSDB / PR / PRP / PTB,1
983174,ANANÁS,2016,1,ELEIÇÕES MUNICIPAIS 2016,TO,92193,92193,13,22022,2.700000e+11,ALLINE RABELO DA SILVA,ALLINE RABELO,VEREADOR,2,DEFERIDO,12,APTO,5,SUPLENTE,22,PR,Partido da República,2.700000e+11,COLIGAÇÃO UNIDOS PELO PROGRESSO I,PR / PSL / PHS / PSDB / PSC / PMB / PC do B / ...,57
983176,ANANÁS,2016,1,ELEIÇÕES MUNICIPAIS 2016,TO,92193,92193,13,12345,2.700000e+11,FABIO COELHO DA SILVA,FABIO COELHO,VEREADOR,2,DEFERIDO,12,APTO,5,SUPLENTE,12,PDT,Partido Democrático Trabalhista,2.700000e+11,ANANÁS EM BOAS MÃOS,PSDC / SD / PDT / PT / PMDB / PSB / PPS / DEM ...,155


In [21]:
local_df["NOME_CANDIDATO"].value_counts(ascending=False)[:30]

LUIZ CARLOS DA SILVA                       820
JOSE CARLOS DA SILVA                       638
JOSÉ CARLOS DA SILVA                       487
ANTONIO CARLOS DA SILVA                    434
LUIZ FERNANDO COSTA                        404
CARLOS EDUARDO DE MATTOS                   388
CARLOS NANTES BOLSONARO                    388
ELISEU GOMES KESSLER                       388
CARLOS ALBERTO CACAU DE BRITO              388
MARCELO BEZERRA CRIVELLA                   388
MARCELO ARAR                               388
ANTONIO CARLOS DOS SANTOS                  388
JORGE MIGUEL FELIPPE                       388
JORGE MAURO DOS SANTOS SILVA               388
MARIA TERESA BERGHER                       388
ROBERTO MONTEIRO SOARES                    388
EDUARDO LOPES MOURA                        388
PEDRO PORFIRIO SAMPAIO                     388
RUBENS ANTONIO ANDRADE COSTA               388
CYRO GARCIA                                388
JAIRO SOUZA SANTOS JUNIOR                  388
TÂNIA CRISTIN

## Mayoral change analysis

In [36]:
local_df_mayors["NOME_URNA_CANDIDATO"].value_counts()[:40]

MARCELO FREIXO        291
CRIVELLA              291
JANDIRA FEGHALI       291
EDUARDO PAES          291
JOSÉ SERRA            200
GABEIRA               196
ANTONIO CARLOS PCO    194
CYRO GARCIA           194
MARTA                 185
FERNANDO HADDAD       174
LEVY FIDELIX          173
PAULINHO              142
SONINHA               119
PEDRO PAULO           117
CELSO RUSSOMANNO      116
GILBERTO KASSAB       114
CARLINHOS             112
SOLANGE               109
CIRO                  104
PAULO RAMOS           102
ANAÍ CAPRONI          100
RODRIGO MAIA          100
LUIZA ERUNDINA        100
OTAVIO LEITE           98
MOLON                  98
CHICO ALENCAR          98
CESAR MAIA             98
CARMEN MIGUELES        97
FILIPE PEREIRA         97
FLÁVIO BOLSONARO       97
CONDE                  97
THELMA MARIA           97
ASPÁSIA                97
FERNANDO SIQUEIRA      97
OCTACILIO RAMALHO      97
ALESSANDRO MOLON       97
NILO BATISTA           97
VINICIUS CORDEIRO      97
OSORIO      

In [33]:
len(local_df_mayors)-len(local_df_mayors.drop_duplicates())

1121

In [38]:
local_df_mayors["DESC_SIT_CAND_TOT"].value_counts()

NÃO ELEITO                                        55863
ELEITO                                            25247
2º TURNO                                           2820
SUBSTITUÍDO                                         230
RENÚNCIA/FALECIMENTO COM SUBSTITUIÇÃO                91
INDEFERIDO COM RECURSO                                3
RENÚNCIA/FALECIMENTO/CASSAÇÃO ANTES DA ELEIÇÃO        1
Name: DESC_SIT_CAND_TOT, dtype: int64

In [37]:
nd_df = local_df_mayors.drop_duplicates()
nd_df["NOME_URNA_CANDIDATO"].value_counts()

EDUARDO PAES       291
CRIVELLA           290
MARCELO FREIXO     289
JANDIRA FEGHALI    286
JOSÉ SERRA         200
                  ... 
NETO MIGUEL          1
GONZAGA BORGES       1
WAL                  1
VICK                 1
FLAVIO REGIS         1
Name: NOME_URNA_CANDIDATO, Length: 37858, dtype: int64

In [22]:
unique_muns = local_df_mayors["NOME_MUNICIPIO"].unique()
print(unique_muns)
print(len(unique_muns))
winning_mayors = []

for mun in unique_muns[:1]:
    print(local_df_mayors[local_df_mayors["NOME_MUNICIPIO"]==mun])

['ACRELANDIA' 'ASSIS BRASIL' 'BRASILEIA' ... 'COUTO MAGALHÃES' 'TABOCÃO'
 'SÃO VALÉRIO']
7681
  NOME_MUNICIPIO  ANO_ELEICAO  NUM_TURNO DESCRICAO_ELEICAO SIGLA_UF  SIGLA_UE  \
0     ACRELANDIA         2004          1     ELEICOES 2004       AC      1120   
1     ACRELANDIA         2004          1     ELEICOES 2004       AC      1120   
2     ACRELANDIA         2004          1     ELEICOES 2004       AC      1120   

   CODIGO_MUNICIPIO  CODIGO_CARGO  NUMERO_CAND  SQ_CANDIDATO  \
0              1120            11           13           6.0   
1              1120            11           14           1.0   
2              1120            11           45          27.0   

                   NOME_CANDIDATO NOME_URNA_CANDIDATO DESCRICAO_CARGO  \
0                 JOÃO DE ALMEIDA               JANGO        PREFEITO   
1  PAULO CÉSAR FERREIRA DE ARAUJO            PAULINHO        PREFEITO   
2     SEBASTIÃO BOCALOM RODRIGUES        TIÃO BOCALOM        PREFEITO   

   COD_SIT_CAND_SUPERIOR DESC_S

In [23]:
local_df_mayors.head()

,NOME_MUNICIPIO,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UF,SIGLA_UE,CODIGO_MUNICIPIO,CODIGO_CARGO,NUMERO_CAND,SQ_CANDIDATO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,DESCRICAO_CARGO,COD_SIT_CAND_SUPERIOR,DESC_SIT_CAND_SUPERIOR,CODIGO_SIT_CANDIDATO,DESC_SIT_CANDIDATO,CODIGO_SIT_CAND_TOT,DESC_SIT_CAND_TOT,NUMERO_PARTIDO,SIGLA_PARTIDO,NOME_PARTIDO,SEQUENCIAL_LEGENDA,NOME_COLIGACAO,COMPOSICAO_LEGENDA,TOTAL_VOTOS
0,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,13,6.0,JOÃO DE ALMEIDA,JANGO,PREFEITO,-1,#NE#,2,DEFERIDO,4,NÃO ELEITO,13,PT,PARTIDO DOS TRABALHADORES,5.0,FRENTE POPULAR DE ACRELÂNDIA,#NE#,1090
1,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,14,1.0,PAULO CÉSAR FERREIRA DE ARAUJO,PAULINHO,PREFEITO,-1,#NE#,2,DEFERIDO,4,NÃO ELEITO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,1.0,#NULO#,#NE#,2060
2,ACRELANDIA,2004,1,ELEICOES 2004,AC,1120,1120,11,45,27.0,SEBASTIÃO BOCALOM RODRIGUES,TIÃO BOCALOM,PREFEITO,-1,#NE#,2,DEFERIDO,1,ELEITO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,12.0,CONTINUANDO O PROGRESSO,#NE#,2173
42,ASSIS BRASIL,2004,1,ELEICOES 2004,AC,1570,1570,11,13,21.0,MANOEL BATISTA DE ARAÚJO,MANOEL,PREFEITO,-1,#NE#,2,DEFERIDO,1,ELEITO,13,PT,PARTIDO DOS TRABALHADORES,4.0,FRENTE POPULAR DE ASSIS BRASIL-FPA,#NE#,1675
43,ASSIS BRASIL,2004,1,ELEICOES 2004,AC,1570,1570,11,23,7.0,ANTONIO BARBOSA DE SOUSA,ZUM,PREFEITO,-1,#NE#,2,DEFERIDO,4,NÃO ELEITO,23,PPS,PARTIDO POPULAR SOCIALISTA,1.0,FRENTE SOCIAL UNIDA - FSU,#NE#,1369


## Ignore below, general utility checkers and random test code

In [24]:
path = os.path.join(DIR, folder_paths[1])
data_path = os.path.join(path, "electoral_data.csv")
test = pd.read_csv(data_path)
test.shape

(646730, 26)

In [25]:
folder_paths

['2004_local',
 '2008_local',
 '2010_fed',
 '2012_local',
 '2014_fed',
 '2016_local']

In [26]:
# column non-overlap testing
ordered_columns = get_ordered_cols("local")
for col in test.columns:
    if col not in ordered_columns:
        print(col)
        
print("-----")
for col in ordered_columns:
    if col not in test.columns:
        print(col)

-----


In [27]:
test.columns

Index(['ANO_ELEICAO', 'NUM_TURNO', 'DESCRICAO_ELEICAO', 'SIGLA_UF', 'SIGLA_UE',
       'CODIGO_MUNICIPIO', 'NOME_MUNICIPIO', 'CODIGO_CARGO', 'NUMERO_CAND',
       'SQ_CANDIDATO', 'NOME_CANDIDATO', 'NOME_URNA_CANDIDATO',
       'DESCRICAO_CARGO', 'COD_SIT_CAND_SUPERIOR', 'DESC_SIT_CAND_SUPERIOR',
       'CODIGO_SIT_CANDIDATO', 'DESC_SIT_CANDIDATO', 'CODIGO_SIT_CAND_TOT',
       'DESC_SIT_CAND_TOT', 'NUMERO_PARTIDO', 'SIGLA_PARTIDO', 'NOME_PARTIDO',
       'SEQUENCIAL_LEGENDA', 'NOME_COLIGACAO', 'COMPOSICAO_LEGENDA',
       'TOTAL_VOTOS'],
      dtype='object')